In [1]:
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn import Sequential, Linear, ReLU, Softmax
from scipy.sparse import random
from scipy import stats
from numpy.random import default_rng
# circuit-sim
# import subcircuit as sc
# import PySpice.Logging.Logging as Logging
# logger = Logging.setup_logging()
# from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
# from PySpice.Unit import *
# from PySpice.Spice.NgSpice.Shared import NgSpiceShared


| Type | Value                          | Type | Value                              |
|------|--------------------------------|------|------------------------------------|
| Node | capacitance value              | Path | input transition time              |
|      | num of input nodes             |      | drive strength of drive cell       |
|      | total input cap                |      | functionality  of drive cell       |
|      | total output cap               |      | drive strength of load cell        |
|      | number of connected resistance |      | functionality of load cell         |
|      | total input resistance         |      | effective capacitance of load cell |
|      | total output resistance        |      | wire path Elmore delay             |
|      | Elmore downstream capacitance  |      | wire path D2M delay                |
|      | Elmore stage delay             |      |                                    |

In [12]:
import networkx as nx
import random

''' Sample Graph '''
# graph configuration
num_nodes = 4
num_edges = 3
num_path_features = 8
num_node_features = 9

# set seed for random number generator
random.seed(42)

# create a linear graph with 4 nodes and 3 edges
G = nx.path_graph(4)

# assign random values to node attributes
for i in range(num_nodes):
    G.nodes[i]['capacitance'] = random.randint(1, 5)
    if i == 0:
        G.nodes[i]['num_input_nodes'] = 0
    else:
        G.nodes[i]['num_input_nodes'] = 1

    G.nodes[i]['input_cap'] = 1
    G.nodes[i]['output_cap'] = 1

    if i == num_nodes - 1:
        G.nodes[i]['connected_resistance'] = 0
    else:
        G.nodes[i]['connected_resistance'] = 1

    G.nodes[i]['input_resistance'] = 1
    G.nodes[i]['output_resistance'] = 1
    G.nodes[i]['elmore_cap'] = 1

    if i == 0:
        G.nodes[i]['elmore_delay'] = 0
    else:
        G.nodes[i]['elmore_delay'] = 1


# get the adjacency matrix
M_adj = nx.adjacency_matrix(G)
print("Adj Matrix: ", M_adj.A)

# extract all paths
paths = nx.all_simple_paths(G, source=0, target=3)
list_of_paths = list(paths)
print("Paths: ", list_of_paths)

# create a matrix of [num_paths, num_path_features]
M_path = torch.zeros(len(list_of_paths), num_path_features)

# assign random value to the paths
for i in range(len(list_of_paths)):
    for j in range(num_path_features):
        M_path[i,j] = random.randint(1, 5)

print("Path Features: ", M_path)

# assign random value to the edges
for i in range(num_nodes):
    for j in range(num_nodes):
        if M_adj.A[i,j] == 1:
            G.edges[i,j]['weight'] = random.randint(1, 5)

# add edge weights to adjacency matrix
for i in range(num_nodes):
    for j in range(num_nodes):
        if M_adj.A[i,j] == 1:
            M_adj.A[i,j] = G.edges[i,j]['weight']

print("Adj Matrix with Edge Weights: ", nx.adjacency_matrix(G).A)

print("Resistance: ", G.edges.data('weight'))

# plot the graph with edge weights
# nx.draw(G, with_labels=True, font_weight='bold')



Adj Matrix:  [[0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 1 0]]
Paths:  [[0, 1, 2, 3]]
Path Features:  tensor([[2., 2., 1., 5., 1., 5., 4., 1.]])
Adj Matrix with Edge Weights:  [[0 1 0 0]
 [1 0 2 0]
 [0 2 0 5]
 [0 0 5 0]]
Resistance:  [(0, 1, 1), (1, 2, 2), (2, 3, 5)]


/home/yuwcse/anaconda3/envs/graph/lib/python3.10/site-packages/scipy/sparse/_base.py:752: VisibleDeprecationWarning: Please use `.todense()` instead
  warn(np.VisibleDeprecationWarning(


$$x_i^{l_1} = ReLU(W_1^{l_1}x_i^{l_1-1}+W_2^{l_1}a_{iu}\sum_{u\in \mathcal{N}(v_i)}x_u^{l_1-1})$$

In [30]:
from torch_geometric.utils import to_dense_adj
# torch random seed
torch.manual_seed(42)

class L1_GNNModule(nn.Module):
    """
    Modified GraphSAGE module with weighted aggregation
    """
    def __init__(self, num_input_features, num_output_features):
        super().__init__()
        self.W1 = nn.Parameter(torch.randn(num_input_features, num_output_features) * 0.01)
        self.W2 = nn.Parameter(torch.randn(num_input_features, num_output_features) * 0.01)

    def forward(self, x, edge_index, edge_weight):
        neighbors_agg = to_dense_adj(edge_index, edge_attr=edge_weight, max_num_nodes=num_nodes).squeeze(0)
        neighbors_agg = torch.matmul(neighbors_agg, x)
        # linear transformation
        out = torch.matmul(neighbors_agg, self.W2) + torch.matmul(x, self.W1)
        # relu activation
        out = torch.relu(out)
        return out

'''Test the GNN Module'''
# input features from node attributes
x = torch.tensor([[G.nodes[i]['capacitance'],
                    G.nodes[i]['num_input_nodes'],
                    G.nodes[i]['input_cap'],
                    G.nodes[i]['output_cap'],
                    G.nodes[i]['connected_resistance'],
                    G.nodes[i]['input_resistance'],
                    G.nodes[i]['output_resistance'],
                    G.nodes[i]['elmore_cap'],
                    G.nodes[i]['elmore_delay']] for i in range(num_nodes)], dtype=torch.float)


# extract edge index from adjacency matrix from networkx
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if M_adj.A[i,j] != 0], dtype=torch.long).t()

# extract edge weights from networkx
edge_weight = torch.tensor([G.edges[i,j]['weight'] for i in range(num_nodes) for j in range(num_nodes) if M_adj.A[i,j] != 0], dtype=torch.float)

gnn = L1_GNNModule(num_node_features, num_node_features)
out = gnn(x, edge_index, edge_weight)

print("Output: ", out)
print("Output Shape: ", out.shape)

Output:  tensor([[0.1052, 0.0341, 0.0045, 0.0000, 0.0177, 0.0043, 0.0000, 0.0000, 0.0225],
        [0.2240, 0.1275, 0.0209, 0.0452, 0.0696, 0.0087, 0.0000, 0.0000, 0.1000],
        [0.3948, 0.2214, 0.1019, 0.1659, 0.1741, 0.0523, 0.0000, 0.0000, 0.1030],
        [0.3837, 0.2295, 0.0763, 0.1510, 0.1672, 0.0361, 0.0000, 0.0000, 0.1748]],
       grad_fn=<ReluBackward0>)
Output Shape:  torch.Size([4, 9])


$$\tilde{a}_{iu}^{(k, l_2)} = softmax(\frac{W_Q^{(k, l_2)}x_i^{(L_1+l_2-1)}(W_k^{(k,l_2)}x_u^{(L_1+l_2-1)})}{\sqrt{d_k}})$$

$$x_i^{(L_1+l_2)} = x_i^{(L_1+l_2-1)} +  W_3^{(l_2)}||^{\mathcal{K}}_{k=1}\sum_{u\in \mathcal{V}}\tilde{a}_{iu}^{(k, l_2)}(W_V^{(k, l_2)}x_u^{(L_1+l_2-1)})$$


In [32]:
class L2_GNNModule(nn.Module):
    """Multi-head Attention GNN module
    """
    def __init__(self, num_input_features, num_output_features):
        super().__init__()
        self.Wq = nn.Parameter(torch.randn(num_input_features, num_output_features) * 0.01)
        self.Wk = nn.Parameter(torch.randn(num_input_features, num_output_features) * 0.01)
        self.Wv = nn.Parameter(torch.randn(num_input_features, num_output_features) * 0.01)
        self.W3 = nn.Parameter(torch.randn(num_input_features, num_output_features) * 0.01)

    def forward(self, x, edge_index=None, edge_weight=None):
        # self-attention map A
        A = torch.matmul(torch.matmul(x, self.Wq), torch.matmul(x, self.Wk).t())
        A = A / torch.sqrt(torch.tensor(num_node_features, dtype=torch.float))
        A = torch.softmax(A, dim=1)

        # Based on the single self-attention map, the node representation of vi can be updated based on aggregating information using a multi-head self-attention map:
        
        # Collaborative Multi-Head Attention
        # concatenate the outputs of all heads
        

'''Test the GNN Module'''
# input features from node attributes
x = torch.tensor([[G.nodes[i]['capacitance'],
                    G.nodes[i]['num_input_nodes'],
                    G.nodes[i]['input_cap'],
                    G.nodes[i]['output_cap'],
                    G.nodes[i]['connected_resistance'],
                    G.nodes[i]['input_resistance'],
                    G.nodes[i]['output_resistance'],
                    G.nodes[i]['elmore_cap'],
                    G.nodes[i]['elmore_delay']] for i in range(num_nodes)], dtype=torch.float)

gnn = L2_GNNModule(num_node_features, num_node_features)
out = gnn(x)

print("Output: ", out)
print("Output Shape: ", out.shape)

    


Output:  tensor([[0.0002, 0.0001, 0.0000, 0.0000, 0.0003, 0.0011, 0.0006, 0.0000, 0.0005],
        [0.0002, 0.0001, 0.0000, 0.0000, 0.0003, 0.0011, 0.0006, 0.0000, 0.0005],
        [0.0002, 0.0001, 0.0000, 0.0000, 0.0003, 0.0011, 0.0006, 0.0000, 0.0005],
        [0.0002, 0.0001, 0.0000, 0.0000, 0.0003, 0.0011, 0.0006, 0.0000, 0.0005]],
       grad_fn=<ReluBackward0>)
Output Shape:  torch.Size([4, 9])
